## 1. Inspecting request_responses.csv file
<p><img src="./nabd.png" style="float: right;" alt="Nabd App" width="200"></p>Nabd App provide easy, fast and reliable way of communication for those who are in need for urgent advice on how to deal with a medical emergency. Trauma specialists call the first 60 minutes after a serious injury "the golden hour". As chances of survival are greatest if the patient gets proper treatment in that hour. This obviously won't replace the need for a doctor, but it helps until medical transport arrives. We are modelling responses to get insights about users' behaviors that will help us improve the app.</p>
<p>We generated a dummy dataset to test the model and once we deploy the app we will link the model to the database to perform our analysis.</p>
<p>The data is stored in <code>datasets/request_responses.csv</code> and it is structured according to RFMTC marketing model (a variation of RFM). </p>
<p> RFM stands for Recency, Frequency and Monetary Value and it is commonly used in marketing for identifying your best customers. In our case, our customers are request responders.
RFMTC is a variation of the RFM model. Below is a description of what each column means in our dataset:</p>
<ul>
<li> R (Recency - time since the last response in hours) </li>
<li> F (Frequency - total number of responses)</li>
<li> M (Monetary - duration of call in minutes)</li>
<li>T (Time - months since the first response)</li>
<li> a binary variable representing whether he/she responded to a call in the last week (1 stands for responding; 0 stands for not responding)</li>
<ul>

In [1]:
# Import pandas
import pandas as pd

# Read in dataset
responses = pd.read_csv('datasets/request_responses3.csv')

# Print out the first rows of our dataset
responses.head()

,Recency (in Hours),Frequency,Monetary(in Minutes),Time(in Weeks),Responded
0,64,7,45,5,1
1,32,10,39,2,1
2,33,16,49,3,1
3,14,10,45,3,1
4,70,14,33,1,1


In [2]:
# Print a concise summary of responses DataFrame
responses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 5 columns):
Recency (in Hours)      245 non-null int64
Frequency               245 non-null int64
Monetary(in Minutes)    245 non-null int64
Time(in Weeks)          245 non-null int64
Responded               245 non-null int64
dtypes: int64(5)
memory usage: 9.7 KB


## 2. Creating target column
<p>We are aiming to predict the value in <code>responded</code> column. We will rename this it to <code>target</code> so that it's more convenient to work with.</p>

In [3]:
# Rename target column as 'target' for brevity 
responses.rename(
    columns={'Responded': 'target'},
    inplace=True
)

# Print out the first 2 rows
responses.head(2)

,Recency (in Hours),Frequency,Monetary(in Minutes),Time(in Weeks),target
0,64,7,45,5,1
1,32,10,39,2,1


## 3. Checking target incidence
<p>We want to predict whether or not the same perosn will respond to a call. The model for this is a binary classifier, meaning that there are only 2 possible outcomes:</p>
<ul>
<li><code>0</code> - the person will not respond to the call</li>
<li><code>1</code> - the person will respond to the call</li>
</ul>
<p>Target incidence is defined as the number of cases of each individual target value in a dataset. That is, how many 0s in the target column compared to how many 1s? Target incidence gives us an idea of how balanced (or imbalanced) is our dataset.</p>

In [4]:
# Print target incidence proportions, rounding output to 3 decimal places
responses.target.value_counts(normalize = True ).round(3)

0    0.571
1    0.429
Name: target, dtype: float64

## 4. Splitting responses into train and test datasets
<p>We'll now use <code>train_test_split()</code> method to split <code>transfusion</code> DataFrame.</p>
<p>Target incidence informed us that in our dataset <code>0</code>s appear 48.6% of the time. We want to keep the same structure in train and test datasets, i.e., both datasets must have 0 target incidence of 48.6%. This can be done using the <code>train_test_split()</code> method from the <code>scikit learn</code> library - by specifiying the <code>stratify</code> parameter. In our case, we'll stratify on the <code>target</code> column.</p>

In [5]:
# Import train_test_split method
from sklearn.model_selection import train_test_split 

# Split responses DataFrame into
# X_train, X_test, y_train and y_test datasets,
# stratifying on the `target` column
X_train, X_test, y_train, y_test = train_test_split(
     responses.drop(columns='target'),
     responses.target,
    test_size=0.2,
    random_state=42,
    stratify= responses['target']
)

# Print out the first 2 rows of X_train
X_train.head(2)

,Recency (in Hours),Frequency,Monetary(in Minutes),Time(in Weeks)
36,53,9,26,4
169,193,3,1,17


## 5. Choosing the best model
<p> We will model data using three methods:
<ul> 
<li> <code> Logistic Regression </code> </li>
<li> <code> Decision Tree </code> </li>
<li> <code> Naive bayes </code> </li>
<ul>
and then choose the best model based on accuracy. </p>

In [1]:
# Naive bayes model
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
NBmodel = BernoulliNB()
NBmodel.fit(X_train, y_train)
y_predict = NBmodel.predict(X_test)
accuracy_score(y_test, y_predict)

ImportError: DLL load failed: The specified module could not be found.

In [ ]:
# Decision Tree model
from sklearn import tree

tree_model = tree.DecisionTreeClassifier()
tree_model.fit(X_train, y_train)
y_predict = tree_model.predict(X_test)
accuracy_score(y_test, y_predict)

In [7]:
# Logistic Regression model
logreg_model = linear_model.LogisticRegression(random_state = 42)
logreg_model.fit(X_train, y_train)
y_predict = logreg_model.predict(X_test)
accuracy_score(y_test, y_predict)

/home/abdulrahman/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7755102040816326

## 6. Go for logistic regression!
<p> We find that logistic regression gives the best accuracy : 77.6%  </p>
<p> We can get more insights about the data using confusion matrix  </p>

In [8]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(
    confusion_matrix(y_test, y_predict),
    columns=['Predicted Not Responded', 'Predicted Responded'],
    index=['True Not Responded', 'True Responded']
)

,Predicted Not Responded,Predicted Responded
True Not Responded,19,9
True Responded,2,19
